# LOTUS demo

## APIs

In [1]:
import bigframes.pandas as bpd
from bigframes.ml.llm import GeminiTextGenerator, _GEMINI_1P5_FLASH_001_ENDPOINT, _GEMINI_1P5_PRO_001_ENDPOINT

data = {
    "Course Name": [
        "Probability and Random Processes",
        "Optimization Methods in Engineering",
        "Digital Design and Integrated Circuits",
        "Computer Security",
        "Operating Systems and Systems Programming",
        "Compilers",
        "Computer Networks",
        "Deep Learning",
        "Graphics",
        "Databases",
        "Art History",
    ]
}
df = bpd.DataFrame(data)
 
model = GeminiTextGenerator(model_name="gemini-1.5-flash-001")

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/global_session.py:113: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return func(get_global_session(), *args, **kwargs)


### 1. `sem_filter`

In [2]:
predict_df = df.sem_filter("{Course Name} requires a lot of math", model, logprobs=True)
predict_df

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,Course Name,primary_results,primary_confidence_scores,verdict_result
0,Probability and Random Processes,True,0.95,True
1,Optimization Methods in Engineering,True,0.95,True
2,Digital Design and Integrated Circuits,True,0.95,True
5,Compilers,True,0.8,True
7,Deep Learning,True,0.95,True
8,Graphics,True,0.8,True


### 2. `sem_join`

In [3]:
import bigframes.pandas as bpd
from bigframes.ml.llm import GeminiTextGenerator

data = {
    "Course Name": [
        "Probability and Random Processes",
        "Optimization Methods in Engineering",
        "Digital Design and Integrated Circuits",
        "Computer Security",
        "Operating Systems and Systems Programming",
        "Compilers",
        "Computer Networks",
        "Deep Learning",
        "Graphics",
        "Databases",
        "Art History",
    ]
}
df = bpd.DataFrame(data)
 
model = GeminiTextGenerator(model_name="gemini-1.5-flash-001")

skills_df = bpd.DataFrame({"Skill": ["Art", "Cryptography", "Baking"]})

join_df = df.sem_join(skills_df, "Taking {Course Name} will make me better at {Skill}", primary_model=model, logprobs=True)
join_df

## Optimizations

### Cascade Models

In [3]:
from bigframes.ml.llm import _GEMINI_1P5_FLASH_001_ENDPOINT, _GEMINI_1P5_PRO_001_ENDPOINT
primary_model = GeminiTextGenerator(model_name=_GEMINI_1P5_FLASH_001_ENDPOINT)
 # Define as backup model for confidence cascade
backup_model = GeminiTextGenerator(model_name=_GEMINI_1P5_PRO_001_ENDPOINT)

In [5]:
predict_df = df.sem_filter("{Course Name} requires a lot of math", primary_model, backup_model, logprobs=True)
predict_df

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,Course Name,primary_results,primary_confidence_scores,backup_results,backup_confidence_scores,verdict_result
0,Probability and Random Processes,True,0.95,<NA>,<NA>,True
1,Optimization Methods in Engineering,True,0.95,<NA>,<NA>,True
2,Digital Design and Integrated Circuits,True,0.95,<NA>,<NA>,True
7,Deep Learning,True,0.95,<NA>,<NA>,True
